In [92]:
from openpyxl import load_workbook
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string, get_column_letter
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np

from filtering import *
from loading import *

PATH = 'SAMPLES\zz_lista_volontera_24_25.xlsx'

wb = load_workbook(PATH, data_only=True)
ws = wb[wb.sheetnames[2]]


column_tuples = []

for col in ws.iter_cols(min_row=7, max_row=9):
    values = [cell.value for cell in col]

    if isinstance(values[1], (int, float)) or isinstance(values[2], (int, float)):
        column_tuples.append(tuple(values))
        
print(column_tuples)

[('1.10.2024.', 7, 6), ('3.10.2024.', 7, 10), ('8.10.2024.', 9, 9), ('10.10.2024.', 9, 10), ('15.10.2024.', 13, 14), ('17.10.2024.', 7, 9), ('22.10.2024.', 8, 9), ('24.10.2024.', 7, 13), ('29.10.2024.', 6, 10), ('31.10.2024.', 5, 5), ('05.11.2024.', 8, 8), ('07.11.2024.', 15, 14), ('12.11.2024.', 12, 17), ('14.11.2024.', 13, 18), ('19.11.2024.', 13, 14), ('21.11.2024.', 11, 15), ('26.11.2024.', 9, 18), ('28.11.2024.', 9, 14), ('3.12.2024.', 9, 11), ('5.12.2024.', 6, 10), ('10.12.2024.', 11, 19), ('12.12.2024.', 6, 17), ('17.12.2024.', 7, 15), ('19.12.2024.', 2, 3), ('7.1.2025.', 5, 11), ('9.1.2025.', 5, 10), ('14.1.2025.', 12, 23), ('16.1.2025.', 10, 16), ('21.1.2025.', 11, 24), ('23.1.2025.', 10, 25), ('28.1.2025.', 12, 17), ('30.1.2025.', 12, 21), ('6.2.2025.', 11, 22), ('11.2.2025.', 13, 29), ('13.2.2025.', 9, 13), ('18.2.2025.', 16, 34), ('20.2.2025.', 25, 26), ('4.3.2025.', 14, 24), ('6.3.2025.', 19, 22), ('11.3.2025.', 9, 18), ('13.3.2025.', 15, 23), ('18.3.2025.', 23, 22), ('20.

<>:11: SyntaxWarning: invalid escape sequence '\z'
<>:11: SyntaxWarning: invalid escape sequence '\z'
C:\Users\Lobel\AppData\Local\Temp\ipykernel_29088\505419652.py:11: SyntaxWarning: invalid escape sequence '\z'
  PATH = 'SAMPLES\zz_lista_volontera_24_25.xlsx'


In [93]:
from cProfile import label
from tkinter import font
from turtle import color
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import matplotlib.dates as mdates


data = column_tuples

# Unpack and convert dates
dates = [datetime.strptime(d[0], "%d.%m.%Y.") for d in data]
volunteer_hours = [d[1] for d in data]
attendees = [d[2] for d in data]

# Convert dates to numbers for fitting
date_nums = [d.toordinal() for d in dates]

# Reference date
split_date = datetime.strptime("09.02.2025.", "%d.%m.%Y.")
split_num = split_date.toordinal()

# Split data for fitting
before_mask = [num < split_num for num in date_nums]
after_mask = [num >= split_num for num in date_nums]

# *************************

# Create figure and axis
fig, ax = plt.subplots(figsize=(14, 7), dpi=800)
plt.tight_layout()

# Helper function to split and fit
def fit_and_plot_trend(ax, date_nums, values, mask, color, label_base, style):
    if sum(mask) >= 2:
        x = np.array([d for d, m in zip(date_nums, mask) if m])
        y = np.array([v for v, m in zip(values, mask) if m])
        poly = np.poly1d(np.polyfit(x, y, 1))

        x_range = np.linspace(min(x), max(x), 100)
        y_fit = poly(x_range)
        fit_dates = [datetime.fromordinal(int(d)) for d in x_range]
        ax.plot(fit_dates, y_fit, linestyle=style, color=color, alpha=0.6, label=f'{label_base} trend')

# Plot original data
ax.plot(dates, volunteer_hours, label='Dolazak volontera', linestyle='-', marker='o', color='blue')
ax.plot(dates, attendees, label='Dolazak djece', linestyle='--', marker='x', color='red')

# Plot split trend lines
fit_and_plot_trend(ax, date_nums, volunteer_hours, before_mask, 'blue', 'Volonteri (prije)', '-')
fit_and_plot_trend(ax, date_nums, volunteer_hours, after_mask, 'blue', 'Volonteri (poslije)', '-')

fit_and_plot_trend(ax, date_nums, attendees, before_mask, 'red', 'Djeca (prije)', '-')
fit_and_plot_trend(ax, date_nums, attendees, after_mask, 'red', 'Djeca (poslije)', '-')

# Add vertical reference line
ax.axvline(split_date, color='orange', linestyle=':', label='Odlazak GGG', linewidth=2)

# Labels and legend
ax.set_xlabel('Datum', labelpad=20, fontsize=14)
ax.set_ylabel('Broj', labelpad=20, fontsize=14)
ax.set_title('Termini, MIOC, 14.4.2025.', fontsize=20, pad=20)
ax.legend()
ax.grid(True)
fig.autofmt_xdate()

# Set proper date formatting on x-axis
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%Y'))
fig.autofmt_xdate()

fig.tight_layout()

fig.savefig('./REZULTATI/4_mjesec/termini/MIOC_termini.png', dpi=800)

# Show the plot
plt.show()
